In [1]:
import magma as m
import inspect
import logging
logging.basicConfig(level=logging.DEBUG)
import fault
from hwtypes import BitVector as BV

In [2]:
@m.circuit.combinational
def basic_if(I: m.Bits[2], S: m.Bit) -> m.Bit:
    if S:
        x = I[0]
    else:
        x = I[1]
    return x


class Main(m.Circuit):
    io = m.IO(I0=m.In(m.Bit), I1=m.In(m.Bit), S=m.In(m.Bit), O=m.Out(m.Bit))
    io.O @= basic_if(m.bits([io.I0, io.I1]), io.S)

m.compile("build/basic_if", basic_if.circuit_definition)
with open('.magma/basic_if.py', 'r') as f:
    print(f.read())

import magma as m
from mantle import mux as phi


class basic_if(m.Circuit):
    io = m.IO(I=m.In(m.Bits[2]), S=m.In(m.Bit), O=m.Out(m.Bit))
    x_0 = io.I[0]
    x_1 = io.I[1]
    x_2 = phi([x_1, x_0], io.S)
    __magma_ssa_return_value_0 = x_2
    O = __magma_ssa_return_value_0
    m.wire(O, io.O)



In [3]:
from magma.simulator import PythonSimulator

sim = PythonSimulator(Main)
sim.set_value(Main.I0, 0)
sim.set_value(Main.I1, 1)
sim.set_value(Main.S, 0)
sim.evaluate()
assert sim.get_value(Main.O) == 1
sim.set_value(Main.S, 1)
sim.evaluate()
assert sim.get_value(Main.O) == 0

In [4]:
tester = fault.Tester(Main)
tester.circuit.I0 = 0
tester.circuit.I1 = 1
tester.circuit.S = 0
tester.eval()
tester.circuit.O.expect(1)
tester.circuit.S = 1
tester.eval()
tester.circuit.O.expect(0)
tester.compile_and_run("verilator")

INFO:root:Running tester...
INFO:root:Success!


In [5]:
from magma.simulator.coreir_simulator import CoreIRSimulator

sim = CoreIRSimulator(Main, None)
sim.set_value(Main.I0, 0)
sim.set_value(Main.I1, 1)
sim.set_value(Main.S, 0)
sim.evaluate()
assert sim.get_value(Main.O) == 1
sim.set_value(Main.S, 1)
sim.evaluate()
assert sim.get_value(Main.O) == 0

In [6]:
from mantle import Not

@m.circuit.combinational
def invert(a: m.Bit) -> m.Bit:
    return Not()(a)

print(repr(invert.circuit_definition))

invert = DefineCircuit("invert", "a", In(Bit), "O", Out(Bit))
not_inst0 = not()
wire(invert.a, not_inst0.in)
wire(not_inst0.out, invert.O)
EndCircuit()


In [7]:
from magma.simulator import PythonSimulator

Invert = invert.circuit_definition

sim = PythonSimulator(Invert)
sim.set_value(Invert.a, 0)
sim.evaluate()
assert sim.get_value(Invert.O) == 1
sim.set_value(Invert.a, 1)
sim.evaluate()
assert sim.get_value(Invert.O) == 0

In [8]:
from magma.simulator.coreir_simulator import CoreIRSimulator

sim = CoreIRSimulator(Invert, None)
sim.set_value(Invert.a, 0)
sim.evaluate()
assert sim.get_value(Invert.O) == 1
sim.set_value(Invert.a, 1)
sim.evaluate()
assert sim.get_value(Invert.O) == 0

In [9]:
tester = fault.Tester(invert.circuit_definition)
tester.circuit.a = 1
tester.eval()
tester.circuit.O.expect(0)
tester.circuit.a = 0
tester.eval()
tester.circuit.O.expect(1)
tester.compile_and_run("verilator")
tester.compile_and_run("python")
tester.compile_and_run("coreir")

INFO:root:Running tester...
INFO:root:Success!
INFO:root:Running tester...
INFO:root:Success!
INFO:root:Running tester...
INFO:root:Success!


In [10]:
@m.circuit.sequential
class Counter:
    def __init__(self):
        self.count : m.UInt[16] = 0

    def __call__(self, inc : m.Bit) -> m.UInt[16]:
        if inc:
            self.count = self.count + 1

        O = self.count
        return O


m.compile("Counter", Counter, output="coreir-verilog")
!coreir -i Counter.json -p instancecount -l commonlib

/Users/leonardtruong/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):
An instance count of all the primitives
invert | instances in current | instances in children | 
  corebit_not | 1 | 0

basic_if | instances in current | instances in children | 
  coreir_mux__width1 | 1 | 0

Mux2xOutUInt16 | instances in current | instances in children | 

Mux2xOutBit | instances in current | instances in children | 
  coreir_mux__width1 | 1 | 0

Main | instances in current | instances in children | 
  coreir_mux__width1 | 1 | 0

Counter_comb | instances in current | instances in children | 
  coreir_add__width16 | 1 | 0
  coreir_const__width16 | 1 | 0

Counter | instances in current | instances in children | 
  coreir_add__width16 | 0 | 1
  coreir_const__widt

In [15]:
from magma.simulator import PythonSimulator

sim = PythonSimulator(Counter, Counter.CLK)
sim.set_value(Counter.inc, True)
sim.evaluate()
for i in range(4):
    print(sim.get_value(Counter.O))
    assert sim.get_value(Counter.O) == i + 1
    sim.advance_cycle()
sim.set_value(Counter.inc, False)
sim.evaluate()
for i in range(4):
    print(sim.get_value(Counter.O))
    assert sim.get_value(Counter.O) == 4
    sim.advance_cycle()

1
2
3
4
4
4
4
4


In [16]:
# TODO: CoreIR simulator should return BV

from hwtypes import BitVector as BV

sim = CoreIRSimulator(Counter, Counter.CLK)
sim.set_value(Counter.inc, 1)
sim.evaluate()
for i in range(4):
    print(sim.get_value(Counter.O))
    assert sim.get_value(Counter.O) == BV[16](i + 1)
    sim.advance_cycle()
sim.set_value(Counter.inc, 0)
sim.evaluate()
for i in range(4):
    print(sim.get_value(Counter.O))
    assert sim.get_value(Counter.O) == BV[16](4)
    sim.advance_cycle()

[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
